## **A Python Implementation of Balanced Risk Set Matching**  
### *By Jyreneah Angel and Nicole Grace Joligon*  
---
### **Abstract**
This programming assignment focuses on the implementation of Balanced Risk Set Matching (BRSM), a statistical technique used in observational studies to control for confounding variables by matching treated and control individuals with similar baseline characteristics. The assignment is part of a Data Analytics course designed to test algorithmic thinking and practical coding skills.

In this assignment, we are tasked with reading the journal article provided and developing a Python implementation of the BRSM methodology. The study referenced in the article explores treatment outcomes for interstitial cystitis, using cystoscopy and hydrodistention, comparing patients with similar symptoms but different timelines for receiving treatment.



### **Introduction**

The study *"Balanced Risk Set Matching"* by *Yunfei Paul Li, Kathleen J. Propert, and Paul R. Rosenbaum* introduces a method called Balanced Risk Set Matching (BRSM) to improve the reliability of observational studies where randomized trials are not feasible. Specifically, the method is used to compare treatments for interstitial cystitis (IC), a chronic bladder condition.

BRSM works by matching treated patients with untreated ones who have similar symptom histories, ensuring that comparisons are not biased by future data. Furthermore, the study employs advanced mathematical techniques, such as integer programming, to ensure that matched groups are balanced in key symptoms like pain and urgency.

Statistical tests were then used to assess the treatment’s effect. While the results showed small improvements in some symptoms (like nocturnal voiding frequency), there were no strong improvements in others. In addition, sensitivity analysis indicated that hidden biases could influence the results, making the conclusions about treatment efficacy tentative.

In light of these findings, a Python implementation of the BRSM methodology would enhance the accessibility, efficiency, and reproducibility of observational studies. Python’s widely used libraries for data analysis, optimization, and statistical testing make it a suitable platform for automating the matching process and ensuring transparent analysis. Moreover, this implementation would enable the handling of large datasets, scale to larger patient populations, and integrate with other analytical tools for further analysis. Additionally, it would facilitate sensitivity analysis to address hidden biases, thereby allowing for broader application of the method across various medical conditions and improving the reliability of treatment effect comparisons in the absence of randomized trials.


### **Step-by-Step Implementation of BRSM Methodology**

#### 1. Data Preparation
Before performing the analysis, it's essential to ensure that the data is properly prepared and cleaned. This step involves collecting relevant variables, ensuring that both treated and control groups are represented correctly, and preprocessing the data to account for different units and ranges.
##### **Objective**:
Prepare the data for analysis by ensuring all relevant variables are available and preprocessed.

##### **Data Requirements**:
- **Treated Group**: Patients who received treatment (e.g., cystoscopy and hydrodistention).
- **Control Group**: Patients who did not receive treatment.
- **Covariates**: Pre-treatment symptom data, such as pain, urgency, and nocturnal voiding frequency.
- **Time Points**: The treatment date for the treated group, with symptom data up until treatment time.

##### **Preprocessing**:
- Normalize or scale symptom data to account for different ranges and units.

#### 2. Risk Set Matching
In this section, we will perform the **Risk Set Matching** process. The goal is to pair treated patients with control patients who have similar pre-treatment symptom histories.

##### **Objective**:
Pair treated patients with control patients who have similar pre-treatment symptoms.

##### **Method**:

*Distance Measure*:
   - We will calculate the **Euclidean distance** (or **Mahalanobis distance**) between the treated and control patients' symptom histories. 
   - The closest match based on the smallest distance will be selected.
     
*Matching Process*:
   - For each treated patient, identify an untreated patient who has a similar symptom history up to the point of treatment.
   - The matching ensures that treatment decisions are based only on pre-treatment data, avoiding the introduction of post-treatment bias in the analysis.
     
#### 3. Balanced Matching (Integer Programming)

In this section, we will perform **Balanced Matching** to ensure that the treatment and control groups are as balanced as possible in terms of pre-treatment symptom distributions.

##### **Objective**:
Minimize the imbalance in symptom distributions between treated and control groups.

##### **Method**:

*Optimization*:
   - The goal is to minimize the **multivariate distance** between matched pairs while ensuring balance in key symptoms such as **pain**, **urgency**, and **nocturnal voiding frequency**.
     
*Penalty Function*:
   - A **penalty function** is introduced in the optimization process to prioritize balance in these key symptoms. The penalty function ensures that the matchings do not just minimize the distance between symptom histories, but also enforce balance across multiple symptom variables.
     
*Implementation*:
   - We will use **network flow** or other optimization algorithms (e.g., **integer programming**) to ensure that the best possible matches are made while balancing the symptoms. These algorithms will adjust the matches to minimize the total distance while maintaining symmetry across symptom distributions.

#### 4. Statistical Analysis

In this section, we will perform **Statistical Analysis** to evaluate whether the treatment had a significant impact on the symptoms of patients post-treatment.

##### **Objective**:
Test if the treatment effect significantly improved symptoms post-treatment.

##### **Methods**:

*Wilcoxon Signed-Rank Test*:
   - The **Wilcoxon Signed-Rank Test** is used to assess whether there is a significant difference between the pre-treatment and post-treatment data within each group (treated vs. control).
   - It is a non-parametric test that compares the median of differences between paired observations, making it suitable for non-normally distributed data.

*Sensitivity Analysis*:
   - **Sensitivity analysis** will be performed to evaluate the potential influence of unobserved confounders on the treatment effect. This helps determine if hidden biases may affect the results and the validity of the conclusions.


### **Application of the Risk Set Matching Methodology**
#### Imports

In [131]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import mahalanobis
from scipy.spatial.distance import cdist
from scipy.stats import wilcoxon
from sklearn.preprocessing import StandardScaler
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
import warnings
import seaborn as sns

warnings.filterwarnings("ignore")

np.random.seed(42)

#### Data Preparation

In [134]:
n_treated = 50
n_controls = 150
n_patients = n_treated + n_controls

# Generate symptom data
base_pain = np.random.normal(loc=5, scale=2, size=n_patients).clip(0, 9)
base_urgency = np.random.normal(loc=5, scale=1.5, size=n_patients).clip(0, 9)
base_frequency = np.random.normal(loc=3, scale=1, size=n_patients).clip(0, 9)

# Create treated and control groups
treated = pd.DataFrame({
    'id': np.arange(n_treated),
    'is_treated': True,
    'baseline_pain': base_pain[:n_treated],
    'baseline_urgency': base_urgency[:n_treated],
    'baseline_frequency': base_frequency[:n_treated],
    'pain_at_t': (base_pain[:n_treated] + np.random.normal(1, 0.5, n_treated)).clip(0, 9),
    'urgency_at_t': (base_urgency[:n_treated] + np.random.normal(1, 0.5, n_treated)).clip(0, 9),
    'frequency_at_t': (base_frequency[:n_treated] + np.random.normal(1, 0.5, n_treated)).clip(0, 9),
})

controls = pd.DataFrame({
    'id': np.arange(n_treated, n_patients),
    'is_treated': False,
    'baseline_pain': base_pain[n_treated:],
    'baseline_urgency': base_urgency[n_treated:],
    'baseline_frequency': base_frequency[n_treated:],
    'pain_at_t': (base_pain[n_treated:] + np.random.normal(0, 0.3, n_controls)).clip(0, 9),
    'urgency_at_t': (base_urgency[n_treated:] + np.random.normal(0, 0.3, n_controls)).clip(0, 9),
    'frequency_at_t': (base_frequency[n_treated:] + np.random.normal(0, 0.3, n_controls)).clip(0, 9),
})

# Simulate post-treatment outcomes (3 months and 6 months)
def simulate_outcomes(base_value, treatment_effect, decay_rate):
    return (
        np.clip(base_value - treatment_effect + np.random.normal(0, 1, len(base_value)), 0, 9),
        np.clip(base_value - (treatment_effect * decay_rate) + np.random.normal(0, 1, len(base_value)), 0, 9)
    )

treated['pain_3mo'], treated['pain_6mo'] = simulate_outcomes(treated['pain_at_t'], 1, 0.8)
treated['urgency_3mo'], treated['urgency_6mo'] = simulate_outcomes(treated['urgency_at_t'], 1, 0.8)
treated['frequency_3mo'], treated['frequency_6mo'] = simulate_outcomes(treated['frequency_at_t'], 1, 0.8)

controls['pain_3mo'] = np.clip(controls['pain_at_t'] + np.random.normal(0, 0.5, len(controls)), 0, 9)
controls['pain_6mo'] = np.clip(controls['pain_3mo'] + np.random.normal(0, 0.5, len(controls)), 0, 9)
controls['urgency_3mo'] = np.clip(controls['urgency_at_t'] + np.random.normal(0, 0.5, len(controls)), 0, 9)
controls['urgency_6mo'] = np.clip(controls['urgency_3mo'] + np.random.normal(0, 0.5, len(controls)), 0, 9)
controls['frequency_3mo'] = np.clip(controls['frequency_at_t'] + np.random.normal(0, 0.5, len(controls)), 0, 9)
controls['frequency_6mo'] = np.clip(controls['frequency_3mo'] + np.random.normal(0, 0.5, len(controls)), 0, 9)

all_data = pd.concat([treated, controls], ignore_index=True)


#### Risk Set Matching - *Distance Measure*

In [137]:
def compute_mahalanobis(treated_df, control_df):
    features = ['baseline_pain', 'baseline_urgency', 'baseline_frequency']
    scaler = StandardScaler()
    combined_data = scaler.fit_transform(pd.concat([treated_df[features], control_df[features]]))
    
    treated_scaled = combined_data[:len(treated_df)]
    control_scaled = combined_data[len(treated_df):]
    inv_cov_matrix = np.linalg.pinv(np.cov(combined_data.T))
    
    distances = cdist(treated_scaled, control_scaled, metric='mahalanobis', VI=inv_cov_matrix)
    return distances

distance_matrix = compute_mahalanobis(treated, controls)

#### Risk Set Matching - *Matching Process*

In [127]:
def match_patients(distance_matrix):
    row_ind, col_ind = linear_sum_assignment(distance_matrix)
    matches = list(zip(row_ind, col_ind))
    return matches

matches = match_patients(distance_matrix)

#### Balanced Matching 

In [141]:
def match_patients(distance_matrix):
    row_ind, col_ind = linear_sum_assignment(distance_matrix)
    matches = list(zip(row_ind, col_ind))
    return matches

matches = match_patients(distance_matrix)

#### Statistical Analysis - *Wilcoxon Signed-Rank Test*

In [ ]:
def perform_wilcoxon_test(df, matches):
    treated_outcomes = [df.loc[df['id'] == treated, 'pain_at_t'].values[0] for treated, control in matches]
    control_outcomes = [df.loc[df['id'] == control, 'pain_at_t'].values[0] for treated, control in matches]
    
    if np.all(np.array(treated_outcomes) == np.array(control_outcomes)):
        return 0, 1.0  # No difference, p-value is 1
    else:
        stat, p_value = wilcoxon(treated_outcomes, control_outcomes)
        return stat, p_value

wilcoxon_stat, p_value = perform_wilcoxon_test(all_data, matches)
print(f"Wilcoxon test statistic: {wilcoxon_stat}, P-value: {p_value}")

#### Statistical Analysis - *Sensitivity Analysis*

In [145]:
def sensitivity_analysis(df, matches, gamma_range):
    results = []
    for gamma in gamma_range:
        adjusted_treated = [
            df.loc[df['id'] == treated, 'pain_at_t'].values[0] * (1 + gamma)
            for treated, control in matches
        ]
        control_outcomes = [df.loc[df['id'] == control, 'pain_at_t'].values[0] for treated, control in matches]
        stat, p_value = wilcoxon(adjusted_treated, control_outcomes)
        results.append((gamma, p_value))
    return results

gamma_range = np.linspace(0, 1, 10)
sensitivity_results = sensitivity_analysis(all_data, matches, gamma_range)
print("Sensitivity Analysis Results:")
for gamma, p_val in sensitivity_results:
    print(f"Gamma: {gamma:.2f}, P-value: {p_val:.4f}")

Sensitivity Analysis Results:
Gamma: 0.00, P-value: 0.4548
Gamma: 0.11, P-value: 0.0181
Gamma: 0.22, P-value: 0.0001
Gamma: 0.33, P-value: 0.0000
Gamma: 0.44, P-value: 0.0000
Gamma: 0.56, P-value: 0.0000
Gamma: 0.67, P-value: 0.0000
Gamma: 0.78, P-value: 0.0000
Gamma: 0.89, P-value: 0.0000
Gamma: 1.00, P-value: 0.0000


### **Visualization of Data**